In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import re

In [ ]:
from IPython.display import display, set_matplotlib_formats
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

Place the input files in the correct path

In [ ]:
fake = pd.read_csv(r"C:\Users\devas\Downloads\Fake.csv")
true = pd.read_csv(r"C:\Users\devas\Downloads\True.csv")

In [ ]:
fake['result'] = 1
true['result'] = 0

In [ ]:
trainingdata = pd.concat([fake, true]).reset_index(drop = True)
trainingdata = trainingdata.drop_duplicates(subset='text')
from sklearn.utils import shuffle
trainingdata = shuffle(trainingdata)
trainingdata = trainingdata.reset_index(drop=True)

In [ ]:
trainingdata.drop(["date"],axis=1,inplace=True)
trainingdata.drop(["title"],axis=1,inplace=True)
trainingdata['text'] = trainingdata['text'].apply(lambda x: x.lower())

In [ ]:
# function to remove special characters
def removespecialchar(text):
    specialchar = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(specialchar, '', text)
 
trainingdata['text'] = trainingdata ['text'].apply(lambda x: ' '.join([removespecialchar(word) for word in x.split()]))

In [ ]:
stop_words = set(stopwords.words('english'))
to_remove = ['•', '!', '"', '#', '”', '“', '$', '%', '&', "'", '–', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '…']
stop_words.update(to_remove)
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub('\[[^]]*\]', '', text)
    text = (" ").join([word for word in text.split() if not word in stop_words])
    text = "".join([char for char in text if not char in to_remove])
    return text

trainingdata['text'] = trainingdata['text'].apply(clean_text)

In [ ]:
testingdata = pd.read_csv(r'C:\Users\devas\Desktop\data\test.csv')


In [ ]:
testingdata = testingdata.drop_duplicates(subset='text')
testingdata.dropna(axis=0, how="any", thresh=None, subset=['text'], inplace=True)

In [ ]:
from sklearn.utils import shuffle
data = shuffle(data)
data = data.reset_index(drop=True)

In [ ]:
data.drop(["author"],axis=1,inplace=True)

In [ ]:
data.drop(["title","id"],axis=1,inplace=True)

In [ ]:
data['text'] = data['text'].apply(lambda x: x.lower())

In [ ]:
# function to remove special characters
def removespecialchar(text):
    specialchar = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(specialchar, '', text)
 
data['text'] = data ['text'].apply(lambda x: ' '.join([removespecialchar(word) for word in x.split()]))

In [ ]:
stop_words = set(stopwords.words('english'))
to_remove = ['•', '!', '"', '#', '”', '“', '$', '%', '&', "'", '–', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '…']
stop_words.update(to_remove)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub('\[[^]]*\]', '', text)
    text = (" ").join([word for word in text.split() if not word in stop_words])
    text = "".join([char for char in text if not char in to_remove])
    return text

data['text'] = data['text'].apply(clean_text)

In [ ]:
a = trainingdata['text']
b = trainingdata['result']

In [ ]:
c  = data['text']
d = data['label']

In [ ]:
size_embedding = 200
windows = 2 
min_count = 1 
maxlen = 1000

text_train_splited = [article.split() for article in a]
w2v_model = gensim.models.Word2Vec(sentences = text_train_splited, 
                                   size = size_embedding, 
                                   window = windows, 
                                   min_count = min_count)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train_splited)
text_train_tok = tokenizer.texts_to_sequences(text_train_splited)
word_index = tokenizer.word_index
print('Size of vocabulary: ', len(word_index))

text_train_tok_pad = pad_sequences(text_train_tok, maxlen=maxlen)

In [ ]:
def w2v_to_keras_weights(model, vocab):
    vocab_size = len(vocab) + 1
    weight_matrix = np.zeros((vocab_size, size_embedding))
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix

embedding_vectors = w2v_to_keras_weights(w2v_model, word_index)

In [3]:
def set_model(embedding_vectors):
    model = Sequential()
    model.add(Embedding(embedding_vectors.shape[0], 
                        output_dim=embedding_vectors.shape[1],
                        weights=[embedding_vectors], 
                        input_length=maxlen, 
                        trainable=False))
    model.add(Dropout(0.3))
    model.add(LSTM(units=120))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    return model

"\ndef set_model(embedding_vectors):\n    mode1 = Sequential()\n    model.add(Embedding(embedding_vectors.shape[0], \n                        output_dim=embedding_vectors.shape[1],\n                        weights=[embedding_vectors], \n                        input_length=maxlen, \n                        trainable=False))\n    model.add(Dropout(0.3))\n    model.add(LSTM(100))  # 100 memory cells / Neurons \n    model.add(Dropout(0.3))\n    model.add(Dense(64,activation='relu'))\n    model.add(Dropout(0.3))\n    model.add(Dense(1, activation='sigmoid'))\n    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])\n    return model\n"

In [ ]:
def set_model1(embedding_vectors):
    model1 = Sequential()
    model1.add(Embedding(embedding_vectors.shape[0], 
                        output_dim=embedding_vectors.shape[1],
                        weights=[embedding_vectors], 
                        input_length=maxlen, 
                        trainable=False))
    model1.add(Dropout(0.3))
    model1.add(Bidirectional(LSTM(100)))  # 100 memory cells / Neurons 
    model1.add(Dropout(0.3))
    model1.add(Dense(64,activation='relu'))
    model1.add(Dropout(0.3))
    model1.add(Dense(1, activation='sigmoid'))
    model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    return model1

In [ ]:
model = set_model(embedding_vectors = embedding_vectors)
model.summary()

In [ ]:
model1 = set_model1(embedding_vectors = embedding_vectors)
model1.summary()

In [ ]:
text_train_splited = [article.split() for article in c]
text_test_tok = tokenizer.texts_to_sequences(text_train_splited)
text_test_tok_pad = pad_sequences(text_test_tok, maxlen=maxlen)

In [ ]:
model.fit(text_train_tok_pad, b,validation_data=(text_test_tok_pad,d), epochs=20, batch_size = 64, verbose = 1)

In [ ]:
model1.fit(text_train_tok_pad, b,validation_data=(text_test_tok_pad,d), epochs=20, batch_size = 64, verbose = 1)

In [ ]:
y_pred = model.predict_classes(text_test_tok_pad)

In [ ]:
y_pred1 = model1.predict_classes(text_test_tok_pad)

In [ ]:
confusion_matrix(d,y_pred)

In [ ]:
confusion_matrix(d,y_pred1)

In [ ]:
accuracy_score(d,y_pred)

In [ ]:
accuracy_score(d,y_pred1)

In [ ]:
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
model.save_weights("model1.h5")
print("Saved model to disk")

In [ ]:
def set_model2(embedding_vectors):
    model2 = Sequential()
    model2.add(Embedding(embedding_vectors.shape[0], 
                        output_dim=embedding_vectors.shape[1],
                        weights=[embedding_vectors], 
                        input_length=maxlen, 
                        trainable=False))
    model2.add(Dropout(0.3))
    model2.add(Bidirectional(LSTM(units=120)))
    model2.add(Dropout(0.3))
    model2.add(Dense(1, activation='sigmoid'))
    model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    return model2

In [ ]:
model2 = set_model2(embedding_vectors = embedding_vectors)
model2.summary()

In [ ]:
history = model2.fit(text_train_tok_pad, b, validation_split=0.2, epochs=30, batch_size = 32, verbose = 1)

In [ ]:
def plot_loss_epochs(history):
    epochs = np.arange(1,len(history.history['acc']) + 1,1)
    train_acc = history.history['acc']
    train_loss = history.history['loss']
    val_acc = history.history['val_acc']
    val_loss = history.history['val_loss']

    fig , ax = plt.subplots(1,2, figsize=(7,3))
    ax[0].plot(epochs , train_acc , '.-' , label = 'Train Accuracy')
    ax[0].plot(epochs , val_acc , '.-' , label = 'Validation Accuracy')
    ax[0].set_title('Train & Validation Accuracy')
    ax[0].legend()
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("Accuracy")

    ax[1].plot(epochs , train_loss , '.-' , label = 'Train Loss')
    ax[1].plot(epochs , val_loss , '.-' , label = 'Validation Loss')
    ax[1].set_title('Train & Validation Loss')
    ax[1].legend()
    ax[1].set_xlabel("Epochs")
    ax[1].set_ylabel("Loss")
    fig.tight_layout()
    fig.show()
    
plot_loss_epochs(history)